In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [6]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, pipeline, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForQuestionAnswering, AutoTokenizer
from datasets import Dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import os
from google.colab import drive

In [4]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased', return_token_type_ids=True)
qa_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
os.getcwd()

'/content'

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!ls /content/drive/My\ Drive

'Colab Notebooks'   MPMC		      'TAC(Assignment-22BLC1066).docx'
 DBMS		    niketGirdhar_CV.pdf       'The Untalked Perspective.pdf'
 images.zip	    pytorch-diffusion-master
'Meeting Reports'   qa.csv


In [23]:
file_path = "/content/drive/My Drive/qa.csv"
qa_df = pd.read_csv(file_path)

In [24]:
qa_df.shape

(29989, 5)

In [25]:
qa_df.head()

,question,context,start_positions,end_positions,answer
0,What sare the benifts of the blood brain barrir?,Another approach to brain function is to exami...,56,60,isolated from the bloodstream
1,What is surrounded by cerebrospinal fluid?,Another approach to brain function is to exami...,16,16,brain
2,What does the skull protect?,Another approach to brain function is to exami...,11,11,brain
3,What has been injected into rats to produce pr...,Another approach to brain function is to exami...,153,153,chemicals
4,What can cause issues with how the brain works?,Another approach to brain function is to exami...,93,94,brain damage


In the dataset, the start_position and end_position are based on tokenized index of starting and ending positions of the answer in the sequence.

In [26]:
qa_df.iloc[0]

,0
question,What sare the benifts of the blood brain barrir?
context,Another approach to brain function is to exami...
start_positions,56
end_positions,60
answer,isolated from the bloodstream


In [27]:
bert_tokenizer.decode(bert_tokenizer.encode(qa_df.iloc[0].question, qa_df.iloc[0].context)[56:61])

'isolated from the bloodstream'

In [28]:
qa_dataset = Dataset.from_pandas(qa_df.sample(25000,random_state=42))

qa_dataset = qa_dataset.train_test_split(test_size=0.2)

In [29]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__'],
        num_rows: 5000
    })
})

In [30]:
def preprocess(data):
    return bert_tokenizer(data['question'],data['context'], truncation=True)

qa_dataset = qa_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [31]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['question', 'context', 'start_positions', 'end_positions', 'answer', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [32]:
# freezing all except the last 2 encoders for finw tuning : reduces the trainning time

for name, param in qa_bert.bert.named_parameters():
    if 'encoder.layer.22' in name:
        break
    param.requires_grad = False # disable training

In [33]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [34]:
batch_size = 22
epochs = 2

training_args = TrainingArguments(
    output_dir = "./qa/results",
    num_train_epochs = epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    logging_dir = './qa/logs',
    save_strategy = 'epoch',
    logging_steps = 10,
    evaluation_strategy = 'epoch',
    load_best_model_at_end = True
)

trainer = Trainer(
    model = qa_bert,
    args = training_args,
    train_dataset = qa_dataset['train'],
    eval_dataset = qa_dataset['test'],
    data_collator = data_collator
)

trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nt_ (eshaan-rithesh2023-vit-chennai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 5.905277252197266,
 'eval_model_preparation_time': 0.0054,
 'eval_runtime': 298.6052,
 'eval_samples_per_second': 16.745,
 'eval_steps_per_second': 0.764}

In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,3.753800,3.708950,0.005400
2,3.538600,3.536242,0.005400


TrainOutput(global_step=1820, training_loss=3.808556242303534, metrics={'train_runtime': 3491.0371, 'train_samples_per_second': 11.458, 'train_steps_per_second': 0.521, 'total_flos': 2.4026818025614896e+16, 'train_loss': 3.808556242303534, 'epoch': 2.0})

In [36]:
trainer.evaluate()

{'eval_loss': 3.5362417697906494,
 'eval_model_preparation_time': 0.0054,
 'eval_runtime': 306.5608,
 'eval_samples_per_second': 16.31,
 'eval_steps_per_second': 0.744,
 'epoch': 2.0}

In [37]:
trainer.save_model()

In [38]:
pipe = pipeline('question-answering', './qa/results', tokenizer = bert_tokenizer)

Device set to use cuda:0


In [40]:
pipe("Where Niket lives?", "Niket and his brother live with their parents in Jalalabad West but nowadays he is in Chennai")

{'score': 0.13361267745494843, 'start': 86, 'end': 93, 'answer': 'Chennai'}

In [41]:
princeton = """In 1675, a Quaker missionary from England, encouraged by New Jersey proprietors John Lord
              "Berkeley and Sir George Carteret, arrived to establish a settlement in this area near the
              "Delaware River, which was inhabited by the Lenni-Lenape Indians. The Keith survey of 1685
              "established the western boundary of Middlesex and Somerset Counties and later, the Township
              "of Princeton. Today Keith's Line is recognized as Province Line Road. With the laying of the
              "cornerstone for Nassau Hall in 1754, Princeton began its development as a location for
              "quality education. Nassau Hall was named for William III, Prince of Orange-Nassau. This simple stone
              "edifice was one of the largest public buildings in the colonies and became a model for many other
              "structures in New Jersey and Pennsylvania."""

pipe("What survey led to the founding of Princeton?", princeton)

{'score': 0.003960967529565096,
 'start': 277,
 'end': 385,
 'answer': 'The Keith survey of 1685 \n              "established the western boundary of Middlesex and Somerset Counties'}